# EdgeTTS

https://github.com/rany2/edge-tts

edge-tts is a Python module that allows you to use Microsoft Edge's online text-to-speech service from within your Python code or using the provided edge-tts or edge-playback command.

In [ ]:
%pip install edge-tts pygame

In [49]:
import edge_tts
import os
import pygame
import time

async def generate_edge_tts_audio(text, file_name, voice='en-US-GuyNeural', style='newscast-formal', verbose=False, play=False, overwrite=False):
    communicate = edge_tts.Communicate(text, voice)
    # whether file exists?
    if os.path.exists(file_name):
        if overwrite:
            if verbose:
                print(f'{file_name} exists, overwriting...')
        else:
            if verbose:
                print(f'{file_name} exists, skipping...')
            return
        
    await communicate.save(file_name)
    if play:
        pygame.mixer.init()
        pygame.mixer.music.load(file_name)
        pygame.mixer.music.play()
    if verbose:
        print(f'{file_name} created')
    
    time.sleep(1.5)

['en-US-GuyNeural', 'en-US-AriaNeural']
hello
../audios/hello-us-male.mp3 created
../audios/hello-us-female.mp3 created

heat
../audios/heat-us-male.mp3 created
../audios/heat-us-female.mp3 created

high
../audios/high-us-male.mp3 created
../audios/high-us-female.mp3 created



In [ ]:

voices = ["en-US-GuyNeural", "en-US-AriaNeural", "en-GB-RyanNeural", "en-GB-LibbyNeural"]
regions = ['us', 'us', 'uk', 'uk']
genders = ['male', 'female', 'male', 'female']

# only_us = False
only_us = True
if only_us:
    voices = voices[:2]
    print(voices)

words = """
hello,
heat,
high,
"""

for word in words.strip().split(','):
    print(word)
    for i, voice in enumerate(voices):
        w = word.strip().lower()
        if len(w) > 0:
            filename = f'../audios/{w.replace(" ", "-")}-{regions[i]}-{genders[i]}.mp3'
            await generate_edge_tts_audio(w, filename, voice=voice, verbose=True, overwrite=False, play=True)


In [63]:
def get_openai_tts_audio(text, path, performer='alloy'):
        
        from openai import OpenAI
        from dotenv import load_dotenv
        load_dotenv()
        client = OpenAI(
        )
        
        with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice=performer,
            input=text.strip()
        ) as response:
            response.stream_to_file(path)
        
sentence = "The explanation you gave was clear but I need a more detailed explanation."

# remove all punctuation at the end of sentence,
# replace all spaces and punctuations in the sentence with dash
audio_filename_openai = sentence.translate(str.maketrans(' ,.?!', '-----')).strip().replace("--", "-") + '_openai.mp3'
audio_filename_msedge = sentence.translate(str.maketrans(' ,.?!', '-----')).strip().replace("--", "-") + '_msedge.mp3'
# get_openai_tts_audio(sentence, audio_filename_openai, performer='alloy')
# await generate_edge_tts_audio(sentence, audio_filename_msedge, voice="en-US-GuyNeural", verbose=True, overwrite=True, play=True)

for voice in ["alloy", "nova"]:
    get_openai_tts_audio(sentence, f'../audios/{sentence.replace(" ", "-")}-{voice}.mp3', performer=voice)


In [54]:
from openai import OpenAI
import os
import IPython
from datetime import datetime
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC, TPE1, TALB, TCON
from dotenv import load_dotenv
from pydub import AudioSegment

load_dotenv()
client = OpenAI(
)

def get_openai_tts_audio(text, filename, performer="alloy"):

    # check artwork.png and ending.mp3 files exist
    if not os.path.isfile('Artwork.png') or not os.path.isfile('ending.mp3'):
        print("Either Artwork.png or ending.mp3 file not found.")
        return

    # split the text into lines
    text = markdown_to_text(text).split("\n")
    # remove empty lines
    text = [t for t in text if t]

    for t in text:
        speech_file_path = f'temp-{text.index(t)}.mp3'
        rspd_audio = client.audio.speech.create(
            model="tts-1",
            voice=performer,
            input=t.strip()
        ) 
        rspd_audio.stream_to_file(speech_file_path)
        # output a progress percentage 
        # keep updating within a line
        print(f"\rprocessing: {round((text.index(t)+1)/len(text)*100)}%", end='...')
    print("\n")

    # create an audio of 1 second of silence
    temp_audio = AudioSegment.silent(duration=1000)
    for t in text:
        seg = AudioSegment.from_file(f'temp-{text.index(t)}.mp3')
        temp_audio += seg + AudioSegment.silent(duration=1500)
        # delete the temp file
        os.remove(f'temp-{text.index(t)}.mp3')
    temp_audio.export('~temp.mp3', format='mp3')
    speech = AudioSegment.from_file('~temp.mp3')
    ending = AudioSegment.from_file('ending.mp3')
    combined = speech + ending
    os.remove('~temp.mp3')
    if filename:
        # if filename has no extension, add .mp3
        if filename.endswith('.mp3'):
            speech_file_path = filename
        else:
            speech_file_path = f'{filename}.mp3'        
    else:
        speech_file_path = f'{datetime.now().strftime("%Y%m%d_%H%M%S")}_{performer}.mp3'
    combined.export(speech_file_path, format='mp3')
    print(f"Audio file saved as {speech_file_path}")

    image_file = 'Artwork.png'
    artist = 'tts'
    album = 'Daily Speech Training'
    genre = 'SPEECH'

    add_metadata(speech_file_path, image_file, artist, album, genre)
    IPython.display.Audio(speech_file_path)

    return f'{speech_file_path} created successfully.'



# English Voices

* voice = "en-US-GuyNeural" (Male)
* voice = "en-US-AnaNeural" (Female)
* voice = "en-US-AndrewNeural" (Male)
* voice = "en-US-AriaNeural" (Female)
* voice = "en-US-AvaNeural" (Female)
* voice = "en-US-BrianNeural" (Male)
* voice = "en-US-ChristopherNeural" (Male)
* voice = "en-US-EmmaNeural" (Female)
* voice = "en-US-EricNeural" (Male)
* voice = "en-US-GuyNeural" (Male)
* voice = "en-US-JennyNeural" (Female)
* voice = "en-US-MichelleNeural" (Female)
* voice = "en-US-RogerNeural" (Male)
* voice = "en-US-SteffanNeural" (Male)
* voice = "en-GB-LibbyNeural" (Female)
* voice = "en-GB-MaisieNeural" (Female)
* voice = "en-GB-RyanNeural" (Male)
* voice = "en-GB-SoniaNeural" (Female)
* voice = "en-GB-ThomasNeural" (Male)
* voice = "en-AU-NatashaNeural" (Female)
* voice = "en-AU-WilliamNeural" (Male)
* voice = "en-CA-ClaraNeural" (Female)
* voice = "en-CA-LiamNeural" (Male)